In [196]:
# DATA2001 Assignment

from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# Assignment

In [197]:
from sqlalchemy import create_engine, text, types
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import geoalchemy2
import geopandas as gpd
import requests
import shapely.geometry
from shapely.geometry import mapping
import time


credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [198]:
db, conn = pgconnect(credentials)
conn.execute(text("set search_path to public"))

Connected successfully.


### Loading Data into Python from .csv, .txt and .shp

In [199]:
data_path = "data"

population_df = pd.read_csv(f"{data_path}/Population.csv")
businesses_df = pd.read_csv(f"{data_path}/Businesses.csv")
income_df = pd.read_csv(f"{data_path}/sa2_income.csv")
postcode_df = pd.read_csv(f"{data_path}/sa2_postcode.csv")
stops_df = pd.read_csv(f"{data_path}/Stops.txt")

sa2_gdf = gpd.read_file(f"{data_path}/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
primary_catchments = gpd.read_file(f"{data_path}/catchments/catchments_primary.shp")
secondary_catchments = gpd.read_file(f"{data_path}/catchments/catchments_secondary.shp")

### Data cleaning before ingestion into database

### Cleaning population

In [200]:
cleaned_population_df = population_df.copy()

cleaned_population_df.columns = (
    cleaned_population_df.columns
    .str.lower()
    .str.replace('-', '_')
    .str.replace(' ', '_')
)


### Cleaning Businesses

In [201]:
cleaned_businesses_df = businesses_df.copy()
for col in cleaned_businesses_df.select_dtypes(include="object"):
    cleaned_businesses_df[col] = cleaned_businesses_df[col].str.strip()
cleaned_businesses_df.rename(columns={
    "0_to_50k_businesses": "b_0_50k",
    "50k_to_200k_businesses": "b_50k_200k",
    "200k_to_2m_businesses": "b_200k_2m",
    "2m_to_5m_businesses": "b_2m_5m",
    "5m_to_10m_businesses": "b_5m_10m",
    "10m_or_more_businesses": "b_10m_plus",
    "total_businesses": "total"
}, inplace=True)


### Cleaning Income

In [202]:
cleaned_income_df = income_df.copy()

cleaned_income_df.rename(columns={
    "SA2_code": "sa2_code",
    "SA2_name": "sa2_name"
}, inplace=True)
cleaned_income_df['income_earners'].fillna(cleaned_income_df['income_earners'].median(), inplace=True)
cleaned_income_df['median_income'].fillna(cleaned_income_df['median_income'].median(), inplace=True)
cleaned_income_df['mean_income'].fillna(cleaned_income_df['mean_income'].median(), inplace=True)
cleaned_income_df['gini_coefficient'].fillna(cleaned_income_df['gini_coefficient'].median(), inplace=True)

/var/folders/7w/0bgjmph51g3fjv7yfs8y17xc0000gn/T/ipykernel_42220/4119000927.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_income_df['income_earners'].fillna(cleaned_income_df['income_earners'].median(), inplace=True)
/var/folders/7w/0bgjmph51g3fjv7yfs8y17xc0000gn/T/ipykernel_42220/4119000927.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

### Cleaning Postcode

In [203]:
cleaned_postcode_df = postcode_df.copy()

postcode_df.rename(columns={
    "SA2_code": "sa2_code",
    "SA2_name": "sa2_name"
}, inplace=True)

### Cleaning Stops

In [204]:
cleaned_stops_df = stops_df.copy()

cleaned_stops_df.columns = cleaned_stops_df.columns.str.strip().str.lower()

for col in cleaned_stops_df.select_dtypes(include='object'):
    cleaned_stops_df[col] = cleaned_stops_df[col].str.strip()

cleaned_stops_df.drop(columns=['parent_station', 'stop_code', 'platform_code', 'location_type'], inplace=True)


### Cleaning Geodata

In [205]:
def clean_gdf(gdf):
    gdf = gdf.copy()
    gdf.columns = gdf.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

    for col in gdf.select_dtypes(include="object"):
        gdf[col] = gdf[col].str.strip()

    return gdf

cleaned_sa2_gdf = clean_gdf(sa2_gdf)
cleaned_primary_catchment= clean_gdf(primary_catchments)
cleaned_secondary_catchment = clean_gdf(secondary_catchments)

target_crs = "EPSG:4326"
cleaned_sa2_gdf = cleaned_sa2_gdf.to_crs(target_crs)
cleaned_primary_catchment = cleaned_primary_catchment.to_crs(target_crs)
cleaned_secondary_catchment = cleaned_secondary_catchment.to_crs(target_crs)

cleaned_school_catchment = pd.concat([cleaned_primary_catchment, cleaned_secondary_catchment], ignore_index=True)
cleaned_school_catchment.drop(columns=[
    "kindergart", "year1", "year2", "year3", "year4", "year5", "year6",
    "year7", "year8", "year9", "year10", "year11", "year12", "priority"
], inplace=True)
cleaned_school_catchment.rename(columns={
    "use_id": "school_id",
    "use_desc": "school_name",
    "geometry": "geom"  
}, inplace=True)

### Load in Geodata

### 1.2. Ingestion

We now need to ingest all of the collected data into the database

In [206]:
conn.execute(text("""
DROP TABLE IF EXISTS population CASCADE;
CREATE TABLE population (
    sa2_code BIGINT PRIMARY KEY,
    sa2_name TEXT,
    "0_4_people" INTEGER,
    "5_9_people" INTEGER,
    "10_14_people" INTEGER,
    "15_19_people" INTEGER,
    "20_24_people" INTEGER,
    "25_29_people" INTEGER,
    "30_34_people" INTEGER,
    "35_39_people" INTEGER,
    "40_44_people" INTEGER,
    "45_49_people" INTEGER,
    "50_54_people" INTEGER,
    "55_59_people" INTEGER,
    "60_64_people" INTEGER,
    "65_69_people" INTEGER,
    "70_74_people" INTEGER,
    "75_79_people" INTEGER,
    "80_84_people" INTEGER,
    "85_and_over_people" INTEGER,
    total_people INTEGER
);
                  
DROP TABLE IF EXISTS businesses CASCADE;
CREATE TABLE businesses (
    industry_code TEXT,
    industry_name TEXT,
    sa2_code BIGINT,
    sa2_name TEXT,
    b_0_50k INTEGER,
    b_50k_200k INTEGER,
    b_200k_2m INTEGER,
    b_2m_5m INTEGER,
    b_5m_10m INTEGER,
    b_10m_plus INTEGER,
    total INTEGER,
    PRIMARY KEY (sa2_code, industry_code)

);
                  
DROP TABLE IF EXISTS income CASCADE;
CREATE TABLE income (
    sa2_code BIGINT PRIMARY KEY,
    sa2_name TEXT,
    income_earners FLOAT,
    median_income FLOAT,
    mean_income FLOAT,
    gini_coefficient FLOAT
);
                  
DROP TABLE IF EXISTS postcode CASCADE;
CREATE TABLE postcode (
    postcode INTEGER,
    sa2_code BIGINT,
    sa2_name TEXT,
    PRIMARY KEY (postcode, sa2_code)
);
                  
DROP TABLE IF EXISTS stops CASCADE;
CREATE TABLE stops (
    stop_id TEXT,
    stop_name TEXT,
    stop_lat FLOAT,
    stop_lon FLOAT,
    sa2_code BIGINT,
    sa2_name TEXT,
    sa4_name TEXT,
    wheelchair_boarding INTEGER,
    geom geometry(Point, 4326)
);
                  
DROP TABLE IF EXISTS sa2_boundaries CASCADE;
CREATE TABLE sa2_boundaries (
    sa2_code21 TEXT PRIMARY KEY,
    sa2_name21 TEXT NOT NULL,
    sa4_code21 TEXT,
    sa4_name21 TEXT,
    areasqkm21 DOUBLE PRECISION,
    geom geometry(MultiPolygon, 4326)
);
                  
DROP TABLE IF EXISTS school_catchments CASCADE;
CREATE TABLE school_catchments (
    school_id TEXT PRIMARY KEY,
    catch_type TEXT NOT NULL,
    school_name TEXT NOT NULL,
    add_date DATE,
    geom geometry(MultiPolygon, 4326) NOT NULL
);
                  
                  

CREATE INDEX idx_catchments_geom ON school_catchments USING GIST (geom);
"""))
cleaned_population_df.to_sql(
    "population",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',          
    chunksize=1000           
)
cleaned_businesses_df.to_sql(
    "businesses",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_income_df.to_sql(
    "income",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_postcode_df.to_sql(
    "postcodes",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_stops_df.to_sql(
    "stops",
    con=conn,
    if_exists='append',
    index=False,
    method='multi',         
    chunksize=1000          
)
cleaned_school_catchment.set_geometry("geom", inplace=True)
cleaned_sa2_gdf.to_postgis("sa2_boundaries", conn, if_exists="replace", index=False)
cleaned_school_catchment.to_postgis("school_catchments", conn, if_exists="replace", index=False)
conn.commit()


In [207]:
query(conn, """
SELECT *
      FROM sa2_boundaries
      WHERE sa4_code21 = '125'""")


,sa2_code21,sa2_name21,chg_flag21,chg_lbl21,sa3_code21,sa3_name21,sa4_code21,sa4_name21,gcc_code21,gcc_name21,ste_code21,ste_name21,aus_code21,aus_name21,areasqkm21,loci_uri21,geometry
0,125011475,Rookwood Cemetery,0,No change,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.0150,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E6100000010000006800000049BE28ACF7E1...
1,125011582,Auburn - Central,0,No change,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.7326,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E610000001000000C001000007E8D5468AE0...
2,125011583,Auburn - North,0,No change,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,2.0966,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E6100000010000000D0100004539CF4AF5E0...
3,125011584,Auburn - South,0,No change,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,2.4313,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E610000001000000D100000078C490139BE0...
4,125011585,Berala,0,No change,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,2.0865,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E610000001000000810000003FA551E5F9E0...
5,125011586,Lidcombe,0,No change,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4036,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E610000001000000F8000000F91BD84F29E1...
6,125011587,Regents Park,0,No change,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,2.0325,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E610000001000000A1000000F2A9D8DCD7E0...
7,125011709,Silverwater - Newington,1,New,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.9340,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E6100000010000005B0100000DCB11E520E1...
8,125011710,Wentworth Point - Sydney Olympic Park,1,New,12501,Auburn,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.3279,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E61000000100000026020000F92D86B7F1E1...
9,125021477,Ermington - Rydalmere,0,No change,12502,Carlingford,125,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,9.8655,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0103000020E610000001000000B50100005D9DB7E8C9E1...


## Task 2 - SA2 Point of Interests

### Pull all POIs in a given bounding box from the API

In [178]:
def shapely_to_esri_polygon(geom):
    """
    Converts a shapely Polygon or MultiPolygon to ESRI JSON polygon format.
    """
    if geom.geom_type == 'Polygon':
        return {
            "rings": [list(geom.exterior.coords)],
            "spatialReference": {"wkid": 4326}
        }
    elif geom.geom_type == 'MultiPolygon':
        rings = []
        for poly in geom.geoms:
            rings.append(list(poly.exterior.coords))
        return {
            "rings": rings,
            "spatialReference": {"wkid": 4326}
        }
    else:
        raise ValueError("Unsupported geometry type for ESRI Polygon")


def get_pois_in_polygon(geom):
    esri_geom = shapely_to_esri_polygon(geom)

    params = {
        "geometry": json.dumps(esri_geom),
        "geometryType": "esriGeometryPolygon",
        "inSR": "4326",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "returnGeometry": "true",
        "f": "json"
    }

    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"

    try:
        response = requests.post(url, data=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print("Request failed:", e)
        return {}

### Loop Through each SA2 in SA4

In [193]:
def get_sa2_geometries_by_sa4(sa4_code, conn):
    """
    Queries the PostGIS DB for SA2 geometries belonging to a given SA4 code.
    Returns a GeoDataFrame.
    """
    sql = """
        SELECT *, geometry
        FROM cleaned_sa2
        WHERE sa4_code21 = %s
    """
    try:
        # Use tuple (not dict) for params
        gdf = gpd.read_postgis(sql, conn, params=(sa4_code,), geom_col="geometry")
        return gdf
    except Exception as e:
        print("Error loading SA2 geometries:", e)
        return gpd.GeoDataFrame()

In [194]:
def get_pois_for_sa2_bboxes(sa4_code, conn):
    # Query DB instead of passing cleaned_sa2_gdf
    sa2_in_sa4 = get_sa2_geometries_by_sa4(sa4_code, conn)
    print(f"Found {len(sa2_in_sa4)} SA2 regions in SA4 '{sa4_code}'")

    all_pois = []

    for idx, row in sa2_in_sa4.iterrows():
        sa2_code = row['sa2_code21']
        sa2_name = row['sa2_name21']
        bbox_geom = row.geometry.envelope

        print(f"\nQuerying POIs for SA2 code {sa2_code} ({sa2_name}) bounding box...")
        time.sleep(1)

        pois_result = get_pois_in_polygon(bbox_geom)
        features = pois_result.get("features", [])

        print(f"Found {len(features)} POIs in SA2 {sa2_code} bounding box.")

        for f in features:
            attr = f["attributes"]
            attr["sa2_code21"] = sa2_code
            attr["sa2_name21"] = sa2_name
            all_pois.append(attr)

    pois_df = pd.DataFrame(all_pois)
    print(f"\nTotal collected POIs: {len(pois_df)}")
    print(pois_df.head())
    return pois_df
pois_df = get_pois_for_sa2_bboxes('125', conn)

Error loading SA2 geometries: (psycopg2.errors.UndefinedTable) relation "cleaned_sa2" does not exist
LINE 3:         FROM cleaned_sa2
                     ^

[SQL: 
        SELECT *, geometry
        FROM cleaned_sa2
        WHERE sa4_code21 = %s
    ]
[parameters: ('125',)]
(Background on this error at: https://sqlalche.me/e/20/f405)
Found 0 SA2 regions in SA4 '125'

Total collected POIs: 0
Empty DataFrame
Columns: []
Index: []


In [165]:
print(pois_df.head())

Empty DataFrame
Columns: []
Index: []
